# NI Water Quality Data API Testing

During development of [NI Water](https://github.com/andrewbolster/bolster/pull/1009), the dataset changed underfoot and became disconnected from the [OpenDataNI dataset](https://admin.opendatani.gov.uk/dataset/ni-water-customer-tap-authorised-supply-point-results) that drove the mapping between Zones and Postcodes.

## API Migration

The NI Water API has been updated from the old `.ashx` endpoint to a new REST API:

- **Old API**: `https://www.niwater.com/water-quality-lookup.ashx?z={zone_code}` (no longer working)
- **New API**: `https://www.niwater.com/api/water-quality/getitem?p={postcode}` (postcode-based lookup)
- **Multi-address**: `https://www.niwater.com/api/water-quality/getitem?z={zone}&p={postcode}` (for postcodes with multiple addresses)

This notebook demonstrates how to use the updated `bolster.data_sources.ni_water` module with the new API.

In [1]:
import sys
import os
from pathlib import Path

# Get the absolute path to the src directory
src_path = Path("../src").resolve()
sys.path.insert(0, str(src_path))

import bolster
from bolster.data_sources.ni_water import *
import pandas as pd

## Example 1: Get Water Quality by Postcode (Single Address)

Most postcodes have a single water supply zone. The new API allows direct lookup by postcode:

In [2]:
# Get water quality data for a single-address postcode
data = get_water_quality_by_postcode('BT14 7EJ')
print(f"Water Supply Zone: {data['Water Supply Zone']}")
print(f"\nHardness Classification: {data['NI Hardness Classification']}")
print(f"Total Hardness: {data['Total Hardness (mg/l)']} mg/l")
data

Water Supply Zone: Dunore Ballygomartin North

Hardness Classification: Moderately Hard
Total Hardness: 64.1 mg/l


0
Water Supply Zone                                                  Dunore Ballygomartin North
Raw Water Source                            This zone is supplied by the main raw water so...
Zone water quality report (2024 dataset)                                         View report.
Total Hardness (mg/l)                                                                    64.1
Magnesium (mg/l)                                                                          8.5
Potassium (mg/l)                                                                          3.7
Calcium (mg/l)                                                                           50.0
Total Hardness (mg CaCO3/l)                                                             160.5
Clark English Degrees                                                                    11.2
French Degrees                                                                           16.1
German Degrees                                            

## Example 2: Get Water Quality by Postcode (Multiple Addresses)

Some postcodes contain multiple addresses served by different water supply zones. The API detects this and automatically uses the first available zone:

In [3]:
# This postcode has multiple addresses with different zones
# The function will automatically select the first zone (ZS0107)
data_multi = get_water_quality_by_postcode('BT12 4PE')
print(f"Water Supply Zone: {data_multi['Water Supply Zone']}")
print(f"Hardness Classification: {data_multi['NI Hardness Classification']}")
data_multi

Water Supply Zone: Belfast Oldpark
Hardness Classification: Moderately Hard


0
Water Supply Zone                                                             Belfast Oldpark
Raw Water Source                            This zone is supplied by the main raw water so...
Zone water quality report (2024 dataset)                                         View report.
Total Hardness (mg/l)                                                                    62.8
Magnesium (mg/l)                                                                          8.8
Potassium (mg/l)                                                                          3.3
Calcium (mg/l)                                                                           48.3
Total Hardness (mg CaCO3/l)                                                             157.3
Clark English Degrees                                                                    11.0
French Degrees                                                                           15.7
German Degrees                                            

## Example 3: Specify Zone Code for Multi-Address Postcodes

If you need a specific address within a multi-address postcode, you can provide the zone code:

In [4]:
# Get data for a specific zone within the postcode
# BT12 4PE has both ZS0107 and ZS0101 zones available
data_specific = get_water_quality_by_postcode('BT12 4PE', zone_code='ZS0101')
print(f"Water Supply Zone: {data_specific['Water Supply Zone']}")
print(f"Hardness Classification: {data_specific['NI Hardness Classification']}")
data_specific

Water Supply Zone: Dunore Ballygomartin North
Hardness Classification: Moderately Hard


0
Water Supply Zone                                                  Dunore Ballygomartin North
Raw Water Source                            This zone is supplied by the main raw water so...
Zone water quality report (2024 dataset)                                         View report.
Total Hardness (mg/l)                                                                    64.1
Magnesium (mg/l)                                                                          8.5
Potassium (mg/l)                                                                          3.7
Calcium (mg/l)                                                                           50.0
Total Hardness (mg CaCO3/l)                                                             160.5
Clark English Degrees                                                                    11.2
French Degrees                                                                           16.1
German Degrees                                            

## Example 4: Get Water Quality by Zone Code

The zone-based lookup still works, but now internally uses the postcode API by finding a postcode for the zone:

In [5]:
# Get water quality data by zone code (backward compatible)
data_zone = get_water_quality_by_zone('ZS0101')
print(f"Water Supply Zone: {data_zone['Water Supply Zone']}")
print(f"Hardness Classification: {data_zone['NI Hardness Classification']}")
data_zone

Water Supply Zone: Dunore Ballygomartin North
Hardness Classification: Moderately Hard


0
Water Supply Zone                                                  Dunore Ballygomartin North
Raw Water Source                            This zone is supplied by the main raw water so...
Zone water quality report (2024 dataset)                                         View report.
Total Hardness (mg/l)                                                                    64.1
Magnesium (mg/l)                                                                          8.5
Potassium (mg/l)                                                                          3.7
Calcium (mg/l)                                                                           50.0
Total Hardness (mg CaCO3/l)                                                             160.5
Clark English Degrees                                                                    11.2
French Degrees                                                                           16.1
German Degrees                                            

## Example 5: Postcode to Water Supply Zone Mapping

The postcode-to-zone mapping comes from OpenDataNI and is used to look up zones:

In [6]:
from bolster.data_sources.ni_water import POSTCODE_DATASET_URL
from tqdm.auto import tqdm

# use csv dict reader and requests with streaming to from the URL and use TQDM to show progress
import requests
import csv
def stream_csv_from_url(url):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    lines = (line.decode('utf-8') for line in tqdm(response.iter_lines(), desc="Reading CSV lines"))
    reader = csv.DictReader(lines)
    return list(tqdm(reader, desc="Downloading CSV data"))

l = stream_csv_from_url(POSTCODE_DATASET_URL)

/Users/bolster/src/bolster/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Reading CSV lines: 50551it [00:01, 45067.81it/s]


In [7]:
mapping = get_postcode_to_water_supply_zone()
print(f"Total postcodes in mapping: {len(mapping)}")
print(f"Total unique zones: {len(set(mapping.values()))}")
print(f"\nSample mappings:")
for postcode, zone in list(mapping.items())[:5]:
    print(f"  {postcode} -> {zone}")

Total postcodes in mapping: 49006
Total unique zones: 65

Sample mappings:
  BT1 1AA -> ZS0107
  BT1 1AL -> ZS0107
  BT1 1AR -> ZS0107
  BT1 1BB -> ZS0107
  BT1 1BH -> ZS0107


In [8]:
# Find all postcodes for a specific zone
zone_postcodes = {code for code, zone in mapping.items() if zone == 'ZS0101'}
print(f"Postcodes in zone ZS0101: {len(zone_postcodes)}")
print(f"Sample postcodes: {list(zone_postcodes)[:10]}")

Postcodes in zone ZS0101: 775
Sample postcodes: ['BT14 7PW', 'BT13 3HU', 'BT12 6DG', 'BT13 2RD', 'BT13 3AZ', 'BT14 7EJ', 'BT13 3RT', 'BT13 3BN', 'BT12 7DS', 'BT13 1FR']


## Example 6: Get All Water Quality Data

Retrieve water quality data for all valid supply zones (this takes several minutes):

In [9]:
# Get all water quality data (WARNING: slow operation)
df = get_water_quality()
print(f"Total zones with data: {len(df)}")
print(f"\nHardness classification distribution:")
print(df['NI Hardness Classification'].value_counts())
df.head()

KeyboardInterrupt: 

In [ ]:
# View the full dataset
df

## Data Fields

Each water quality record includes the following fields:

In [ ]:
# Show all available data fields
sample_data = get_water_quality_by_postcode('BT14 7EJ')
print("Available fields:")
for field in sample_data.index:
    print(f"  - {field}")